# 第6章：仮説検証型データ分析

## 6.1 売場に立ち寄っただけのショッパーと、商品を購入したショッパーでは、なにが違うだろうか？商品を購入したショッパーにはなにか特徴があるだろうか？

### 【解答例】

まずは以下の通り仮説を立てた。

- 仮説1:売場に立ち寄っただけのショッパー（以降「立寄者」）はエリア立寄回数が少ないが、商品を購入したショッパー（以降「購入者」）はエリア立寄回数が多い
- 仮説2:立寄者はフレーム滞在時間が短いが、購入者はフレーム滞在時間が長い
- 仮説3:立寄者は商品接触回数が多いが、購入者は商品接触回数が少ない

この3つの仮説をデータから検証してみよう。

- 初期処理

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df1 = pd.read_csv("../input/gi_train_mm10.csv") # 10月のデータ
df2 = pd.read_csv("../input/gi_train_mm11.csv") # 11月のデータ

df_tmp = pd.concat([df1, df2])
df = df_tmp[df_tmp["customer_id"].notna()].copy()

- 立寄者：売場に立寄ったが商品は購入していないショッパー
- 購入者：売場に立寄り、商品を購入したショッパー

In [ ]:
buy_customer = set(df[df["buy_flag"]==1]["customer_id"].unique())
tachiyori_customer = set(df["customer_id"].unique()) - buy_customer

buy_customer_df = df[df["customer_id"].isin(buy_customer)]
tachiyori_customer_df = df[df["customer_id"].isin(tachiyori_customer)]

#### 仮説1

- エリア立寄数のカウント方法
  - 一人のショッパーがあるエリアに入って、出て、入って、出てをした場合、エリア立寄数のカウントは「2」とする
  - 仮に、同じエリアの複数回の出入をしたとしてもエリア立寄数のカウントを「1」にするには下記スクリプトの「count()」を「nunique()」に変更する

In [ ]:
# 立寄者
print("立寄者一人あたりのエリア立寄数の平均",tachiyori_customer_df.groupby("customer_id")["area_id"].count().mean())

# 購入者
print("購入者一人あたりのエリア立寄数の平均",buy_customer_df.groupby("customer_id")["area_id"].count().mean())

以上から、立寄者よりも購入者の方がエリア立寄回数が多く、仮説は正しいことが分かった。

#### 仮説2

In [ ]:
# 立寄者
print("立寄者一人あたりのフレーム滞在時間の平均",tachiyori_customer_df[tachiyori_customer_df["event_type"]==1]["time_duration"].mean())

# 購入者
print("購入者一人あたりのフレーム滞在時間の平均",buy_customer_df[buy_customer_df["event_type"]==1]["time_duration"].mean())

以上から、立寄者よりも購入者の方がフレーム滞在時間が長く、仮説は正しいことが分かった。

#### 仮説3

In [ ]:
# 立寄者
print("立寄者一人あたりの商品接触回数の平均",tachiyori_customer_df.groupby("customer_id")["num_touch"].sum().mean())

# 購入者
print("購入者一人あたりの商品接触回数の平均",buy_customer_df.groupby("customer_id")["num_touch"].sum().mean())

以上から、立寄者よりも購入者のほうが商品接触回数が多く、仮説は誤りであることが分かった。

### 【解説】

- 書籍本編でも行ったように、まずは日本語で仮説を立ててみましょう。その際、お題通り「売場に立ち寄っただけのショッパーは○○、商品を購入したショッパーは××」というように対比して書き出して見ると、データ分析にも繋げやすいでしょう。
- 仮説3のように、実際にデータを分析してみたら仮説と異なる結果が出てくることも多くあります。このように、直感とは異なる事実を見つけることが出来るのも仮説検証型データ分析の醍醐味です。

## 6.2 最も併接触される商品の組み合わせはなんだろうか？

### 【解答例】

- ここではまず、どのような導出の仕方をしたら良いかを日本語で書いてみよう

- まずは「最も併接触される商品の組み合わせ」といった時に、いくつの商品の組み合わせを考えるのかで考え方が変わってくる。ここでは簡便化のために「2商品の組み合わせ」を考えてみよう。つまり「最も併接触される商品の組み合わせは『商品Aと商品Bの組み合わせ』である」というのが答え方になる。
- そこで、2つの商品組み合わせを作るスクリプトを書いてみよう。
  - 商品A、商品B
  - 商品A、商品C
  - ・
  - ・
  - ・
  - 商品Y、商品Z
- 次にそれぞれの組み合わせ商品を両方接触している人を特定しよう
  - 商品Aに接触している人は・・・
  - 商品Bに接触している人は・・・
  - 商品Aと商品Bの両方に接触している人は・・・
- 上記を1つのDataFrameにまとめてみよう

In [ ]:
import itertools

In [ ]:
# 商品の組み合わせDataFrameを作成
combis = list(itertools.combinations(df["product_name"].unique(), 2))
base_df = pd.DataFrame(combis, columns=["商品_A","商品_B"])
# 欠損値が含むレコードを削除する
base_df.dropna(how="any",inplace=True)
base_df.head()

In [ ]:
# データフレームを1行ずつ取り出し、商品_Aと商品_Bの両方に接触した人を特定する
for index, data in base_df.iterrows():
    # 商品_Aに接触したショッパー
    customer_A = set(df[(df["product_name"]==data["商品_A"]) & (df["num_touch"]==1)]["customer_id"].unique())
    # 商品_Bに接触したショッパー
    customer_B = set(df[(df["product_name"]==data["商品_B"]) & (df["num_touch"]==1)]["customer_id"].unique())
    # 商品_Aと商品_Bの両方に接触したショッパー
    customer_A_B = customer_A & customer_B
    # データフレームに併接触人数を追加する
    base_df.loc[index,"併接触人数"] = len(customer_A_B)
    
# 併接触人数が多い順に並び替え
base_df.sort_values("併接触人数",ascending=False,inplace=True)

base_df.head()

以上から、「アフタヌーンレモンティー500ml」と「ほっと一息500ml」の組み合わせが、最も併接触される（16人）商品の組み合わせであるといえる。

### 【解説】

- このように、少し複雑なプログラムになりそうな場合は、まずは日本語でその仕様を書き出してみると良いでしょう。
- また「1人の人が同一商品を複数回接触した際にどのようにカウントするか」のような疑問が出てきた場合は、解答例のように具体的な事例を挙げて考えてみると分かりやすいでしょう。

## 6.3 時間帯別に立寄者の性別や年代は異なるだろうか？また、その結果と時間帯別の接触商品にはなにか関係性はありそうだろうか？

### 【解答例】

- まずは「時間帯別に立寄者の性別や年代は異なるだろうか？」について考えてみよう。この問題を以下のふたつの問題にさらに分解してデータ分析で解答をだしてみよう。
    - 時間帯ごとに「最も多い立寄者の性別」を導出する
    - 時間帯ごとに「最も多い立寄者の年代」を導出する
- この2つはそれぞれ表として出せそうで、それを組み合わせると1つの表になりそうだ。このプログラムを作成してみると、以下のようになるだろう。

In [ ]:
# 時間のカラムを作成するために、まずはin_timeをobject -> datetime型に変更する
df["in_time"] = pd.to_datetime(df["in_time"])
# 時間カラムを追加する
df["時間"] = df["in_time"].dt.hour

- 時間帯ごとに「最も多い立寄者の性別」を導出する

In [ ]:
# ピポットテーブルを作成する
gender_df = df[df["event_type"]==1].pivot_table(index="時間",columns="gender",values="customer_id",aggfunc=len)

# データフレームを1行ずつ取り出し、man/womanの多い方を判定カラムに追加する
for index, data in gender_df.iterrows():
    gender_df.loc[index,"判定(性別)"] = data.idxmax()

gender_df

- 時間帯ごとに「最も多い立寄者の年代」を導出する

In [ ]:
age_df = df[df["event_type"]==1].pivot_table(index="時間",columns="age",values="customer_id",aggfunc=len)

# データフレームを1行ずつ取り出し、年代の多い方を判定カラムに追加する
for index, data in age_df.iterrows():
    age_df.loc[index,"判定(年代)"] = data.idxmax()

age_df

- 2つのDataFrameを連結する（判定列のみ）

In [ ]:
gender_age_df = pd.concat([gender_df[["判定(性別)"]],age_df[["判定(年代)"]]],axis=1)
gender_age_df

- このプログラムの結果から「時間帯別に立寄者の性別や年代は異なる」ということがいえそうだ。
- 朝（8時台）や夜（19時台以降）は男性の方が多く、昼間の時間帯は女性の方が多いということも興味深い。
- では次に「その結果と時間帯別の接触商品にはなにか関係性はありそうだろうか？」について考えてみよう。先ほど作成した一覧表の一番右側の列に「その時間帯に最も接触された商品」の列を追加してみよう。

In [ ]:
for index, data in gender_age_df.iterrows():
    # 各時間毎の最も接触された商品
    most_touch_product = df[(df["時間"]==index) & (df["num_touch"]==1)]["product_name"].value_counts().index[0]
    # 先ほどのデータフレームに追加する
    gender_age_df.loc[index,"最も接触された商品"] = most_touch_product

gender_age_df

- この結果から以下のようなことがいえそうだ。
    - 時間帯別に立寄者の最も多い性別や年代は異なる
    - 時間帯別に最も多く接触された商品も異なる
- 一方で、時間帯と性別と年代と最も多く接触された商品に相関があるのか、ということは、ここからはまだなんともいえない。「最も多く接触された商品」というようなデータは数値データではないため、相関係数を算出するのもできそうもない。
- ただし、「最も接触された商品」として最も商品名が挙がった「トラディショナル緑茶525ml」について、時間帯と性別と年代の関係を捉えることはできそうだ。
- 時間帯と性別・年代のピポットテーブルを作成し、それぞれの関係性をみていく。

In [ ]:
import seaborn as sns
import japanize_matplotlib

In [ ]:
# トラディショナル緑茶525mlに接触した人の、性別×年代別の集計
tmp = df[(df["product_name"]=="トラディショナル緑茶525ml") & (df["num_touch"]==1)].pivot_table(index="時間",columns=["age","gender"],values="customer_id",aggfunc=len).fillna(0)
sns.heatmap(tmp,annot=True,cmap="Blues")

- 上記をみると、「トラディショナル緑茶525ml」は15時以降の30代・40代の男女ともに接触が多いことがわかる。
- 時間帯と性別と年代と商品接触回数の関係性がみえてきた。
- 他の商品にも同じような傾向があるか確認する必要がありそうだ。ぜひ試してみよう。

### 【解説】

- 最後の問いかけのように「関係性を出したい」という場合に相関係数以外にどのような解答の出し方があるのか、そのやり方を考えてみると良いでしょう。今回は「トラディショナル緑茶525ml」に着目して、その商品が最も手に取られる時間帯と性別・年代の組み合わせについて、マトリクスにしてヒートマップ形式で表示するという方法をとりましたが、もちろん他にも傾向を掴む方法あるでしょう。
- なお、今回はseabornという可視化ライブラリを使用してヒートマップのような形にしてみました。こうすると、特定の性別・年代・時間帯にその商品の接触回数が多い・少ないといったことが一目で分かります。このような可視化手法を色々知っておくとより精緻にデータと向き合うことができます。